In [1]:
!pip install azure-eventhub

In [37]:
# Import classes from Event Hubs python package
from azure.eventhub import EventHubClient, Receiver, Offset,EventData
import os
import time 
import datetime
import csv
import json
import operator
from IPython.display import display, clear_output


In [12]:
#if it is the first time use below"
#datastartdate=datetime.datetime.strptime("2020-01-01 00:00:00", "%Y-%m-%d %H:%M:%S")
#print(datetime.datetime.strftime(datastartdate, "%Y-%m-%d %H:%M:%S"))
dataadd=datetime.timedelta(days=365)
#later use below or implement reading from a log file where you left
#datastartdate=datetime.datetime.today().replace(hour=0, minute=0, second=0, microsecond=0)
datastartdate=datetime.datetime.strptime("2020-01-03 14:25:19", "%Y-%m-%d %H:%M:%S")
print(datetime.datetime.strftime(datastartdate, "%Y-%m-%d %H:%M:%S"))  
print(dataadd)

2020-01-01 00:00:00
2020-01-01 00:00:00
365 days, 0:00:00


In [6]:
os.getcwd()

'C:\\Users\\power\\Documents\\RanpUp'

In [7]:
os.chdir('Data')

In [8]:
files=sorted(os.listdir())
print(files)

['green_tripdata_2019-01.csv']


In [10]:
#implement here something to get the right date for file
fileName=files[0]

In [47]:
#read into a list of dictioneries
TripList = []
with open(fileName,'r') as csv_file:
    reader = csv.DictReader(csv_file)
    TripList= [r for r in reader]
print(len(TripList))

630918


Check the data

In [48]:
#STripList = sorted(TripList, key=datetime.datetime.strptime(operator.itemgetter('lpep_dropoff_datetime'), "%Y-%m-%d %H:%M:%S"), reverse=False)
STripList = sorted(TripList, key=operator.itemgetter('lpep_dropoff_datetime'), reverse=False).copy()
print(datastartdate)
print(STripList[0]['lpep_dropoff_datetime'])

2020-01-01 00:00:00
2009-01-01 00:06:03


In [49]:
print(STripList[0])
print("")
print(json.dumps(STripListUp[0]))

OrderedDict([('VendorID', '2'), ('lpep_pickup_datetime', '2009-01-01 00:06:03'), ('lpep_dropoff_datetime', '2009-01-01 00:06:03'), ('store_and_fwd_flag', 'N'), ('RatecodeID', '1'), ('PULocationID', '264'), ('DOLocationID', '193'), ('passenger_count', '1'), ('trip_distance', '.00'), ('fare_amount', '0'), ('extra', '0'), ('mta_tax', '0'), ('tip_amount', '0'), ('tolls_amount', '0'), ('ehail_fee', ''), ('improvement_surcharge', '0'), ('total_amount', '0'), ('payment_type', '2'), ('trip_type', '1'), ('congestion_surcharge', '')])

{"VendorID": "2", "lpep_pickup_datetime": "2010-01-01 00:06:03", "lpep_dropoff_datetime": "2010-01-01 00:06:03", "store_and_fwd_flag": "N", "RatecodeID": "1", "PULocationID": "264", "DOLocationID": "193", "passenger_count": "1", "trip_distance": ".00", "fare_amount": "0", "extra": "0", "mta_tax": "0", "tip_amount": "0", "tolls_amount": "0", "ehail_fee": "", "improvement_surcharge": "0", "total_amount": "0", "payment_type": "2", "trip_type": "1", "congestion_surcha

Event hubs conenction info:

In [15]:
# Address can be in either of these formats:
# "amqps://<URL-encoded-SAS-policy>:<URL-encoded-SAS-key>@<mynamespace>.servicebus.windows.net/myeventhub"
# "amqps://<mynamespace>.servicebus.windows.net/myeventhub"
# For example:
ADDRESS = "amqps://rampupentry.servicebus.windows.net/nytaxi"

# SAS policy and key are not required if they are encoded in the URL
USER = "RootManageSharedAccessKeyNYTXpr"
KEY = "H5lA9eaxr/C04KqeSxjXXLDjmFHWBhW++eLEh48oe2M="

Create and start the client

In [50]:
# Create an Event Hubs client
client = EventHubClient(ADDRESS, debug=False, username=USER, password=KEY)
# Add a sender to the client
sender = client.add_sender(partition="0")

# Run the Event Hub client
client.run()

[]

iterate through the code converting the data to 2020. ıf the code fetches the actual time it will send it real time


In [51]:
STripListUp = STripList.copy()
for i in STripListUp:
    trippickdate = datetime.datetime.strptime(i['lpep_pickup_datetime'], "%Y-%m-%d %H:%M:%S")  +dataadd
    tripdropdate = datetime.datetime.strptime(i['lpep_dropoff_datetime'], "%Y-%m-%d %H:%M:%S")  +dataadd
    i['lpep_pickup_datetime'] = trippickdate.strftime("%Y-%m-%d %H:%M:%S") 
    i['lpep_dropoff_datetime'] = tripdropdate.strftime("%Y-%m-%d %H:%M:%S")

    datafromoffset=(tripdropdate - datastartdate).total_seconds()
  
    if datafromoffset>= 0:
        secs=(tripdropdate - datetime.datetime.now()).total_seconds()
                                                      
        if secs<=0:
            clear_output(wait=True)
            display('Sending message for a trip happened at '+ i['lpep_dropoff_datetime']) 
            sender.send(EventData(json.dumps(i)))
            #time.sleep(2)
        elif secs>86400:
            #it is a day.
            clear_output(wait=True)
            display('Next data will come a day later after '+str(secs)+' seconds, hence breaking')
            #implement here something like a log file to keep where i left
            break          
        
        else:
            clear_output(wait=True)
            display('Will wait for '+str(secs)+' for the next event send') 
            display('Sending message for a trip will happen  at '+ i['lpep_dropoff_datetime']) 
            time.sleep(secs)
            #print(json.dumps(i))
            sender.send(EventData(json.dumps(i)))
    else:
        clear_output(wait=True)
        display('Trip date too small : '+i['lpep_dropoff_datetime'] +' will skip sending')     
          

            
  
                                                
        

'Will wait for '

'1.964153'

'for the next event send'

KeyboardInterrupt: 

In [54]:
# Stop the Event Hubs client
client.stop()